In [ ]:
from bottle import Bottle,run,template,request,redirect
import sqlite3

app=Bottle()

@app.route('/juegos',method='GET')
def juego():
    connection=sqlite3.connect('todo.db')
    cursor=connection.cursor()
    cursor.execute("SELECT DISTINCT tittle FROM game_list")
    list_titles=cursor.fetchall()
    value=[]
    for title in list_titles:
        value.append(title[0])
    dict_titles={}
    dict_titles["titles"]=value
    return str(dict_titles)
    
    #list_titles={'titles':['title','title','title']}
    #return list_titles

@app.route('/games/date/:date',method='GET')
def get_list_games(date):
    connection=sqlite3.connect('todo.db')
    cursor=connection.cursor()
    cursor.execute(f"SELECT tittle,price FROM game_list WHERE date='{date}'")
    list_titles=cursor.fetchall()
    titles=[]
    for title in list_titles:
        temp_dict={title[0]:title[1]}
        titles.append(temp_dict)
    dict_titles={}
    dict_titles["titles"]=titles
    
    output=template('table',rows=dict_titles)
    return output

@app.route('/games/date/',method='GET')
def get_list_games():
    data=request.json
    connection=sqlite3.connect('todo.db')
    cursor=connection.cursor()
    date=data["date"]
    cursor.execute(f"SELECT tittle,price FROM game_list WHERE date='{date}'")
    list_titles=cursor.fetchall()
    list_games=[]
    for title in list_titles:
        gameobj = {title[0]:title[1]}
        list_games.append(gameobj)
    return {date:list_games}

@app.route('/games/history',method='GET')
def get_game_history():
    data = request.json
    connection=sqlite3.connect('todo.db')
    cursor=connection.cursor()
    title_game=data["title"]
    cursor.execute(f"SELECT price,date FROM game_list WHERE tittle='{title_game}'")
    list_titles=cursor.fetchall()
    list_prices=[]
    for title in list_titles:
        gameobj = {title[0]:title[1]}
        list_prices.append(gameobj)
    
    return {title_game:list_prices}

@app.route('/games/update/price',method='POST')
def update_game_price():
    data=request.json
    connection=sqlite3.connect('todo.db')
    cursor=connection.cursor()
    title_game=data["title"]
    title_price=data["price"]
    title_date=data["date"]
    cursor.execute(f"UPDATE game_list SET price='{title_price}' WHERE tittle='{title_game}' AND date='{title_date}'")
    connection.commit()
    cursor.execute(f"SELECT price,date FROM game_list WHERE tittle='{title_game}' AND date='{title_date}'")
    list_titles=cursor.fetchall()
    list_games=[]
    for title in list_titles:
        gameobj = {"Precio":title[0],"Fecha":title[1]}
        list_games.append(gameobj)
    cursor.close()
    
    return {title_game:list_games}


run(app,host='0.0.0.0',port=5000,debug=True)